#  Stationary Markov Chain Class — Notes

---

## **1. Objective**

We are asked to create a **Python class** representing a **stationary Markov chain**.  
Each instance of this class corresponds to one particular chain characterized by:

- Dimension of the state space $K$  
- Transition matrix $Q$  
- Initial probability distribution $\pi$

---

## **2. Class Specification**

### **Constructor**

The class should be named `MarkovChain`.

It takes the following arguments:

| Argument | Type | Description | Default |
|-----------|------|--------------|----------|
| `K` | int | Dimension of the state space | *(required)* |
| `Q` | 2-D NumPy array $(K\times K)$ | Transition matrix; each row is a probability vector (row sums to 1) | Constant matrix with entries $1/K$ |
| `pi` | 1-D NumPy array (length $K$) | Initial probability vector | Uniform vector (entries $1/K$) |

---

### **Attributes**

After initialization, the object should have:

- `self.K` → dimension of the chain  
- `self.Q` → transition matrix  
- `self.pi` → initial distribution (probability vector)

---

## **3. Required Methods**

### **(a) realization(n)**

- Input: integer `n` (number of steps)  
- Output: NumPy array of length `n` containing a **realization**  
  $$
  (X_0, X_1, \ldots, X_{n-1})
  $$
  Each element should be an integer index in $\{0,1,\ldots,K-1\}$.

**Algorithm outline**

1. Initialize `X[0]` by sampling from `pi`.  
2. For each $t=1,\ldots,n-1$:
   - Sample `X[t]` using the conditional distribution given `X[t-1]` (row of `Q`).
3. Return the simulated path as a NumPy integer array.

---

### **(b) distribution(n)**

- Input: integer `n`  
- Output: probability vector (1-D NumPy array) representing the distribution of $X_n$.

**Computation**

Let $\pi^{(0)} = \pi$ (the initial distribution).  
Then, by the Markov property:

$$
\pi^{(n)} = \pi Q^n.
$$

Return $\pi^{(n)}$ as the result.

---

### **(c) stationary_distribution()**

- Output: stationary distribution $v$ satisfying:

$$
vQ = v, \qquad \sum_i v_i = 1, \quad v_i \ge 0.
$$

**How to compute**

1. Solve the linear system $(Q^T - I)v^T = 0$ with the constraint $\sum_i v_i = 1$.  
2. Alternatively, find the **left eigenvector** of $Q$ corresponding to eigenvalue 1 and normalize it so entries sum to 1.  
3. Ensure all values are real (discard small imaginary parts from numerical rounding).

---

## **4. Testing the Implementation**

The following script tests correctness and basic properties of the class.


In [59]:
import numpy as np
class MarkovChain:
    def __init__(self,K,Q=None,pi=None):
        if Q is None:
            self.Q=np.ones((K,K))/K
        else:
            self.Q=Q
        if pi is None:
            self.pi=np.ones(K)/K
        else:
            self.pi=pi
    def realization(self,n):
        X=np.zeros(n,dtype=int)
        X[0]=np.random.choice(range(K),p=self.pi)
        for i in range(1,n):
            X[i]=np.random.choice(range(K),p=self.Q[X[i-1],:])
        return(X) 
    def distribution(self,n):
        p=self.pi
        for i in range(n):
            p=p@self.Q
        return(p)
    def stationary_distribution(self):
        e=np.linalg.eig(Q.T)
        v=e[1][:,0]
        v=v/sum(v)
        v=np.real(v)
        return(v)
#
# create a Markov chain instance
# using a randomly generated transition matrix
K=5
Q=np.random.uniform(0,1,(K,K))
for i in range(K):
    Q[i,:]=Q[i,:]/sum(Q[i,:])
m=MarkovChain(5,Q=Q)
print("Q  = ")
print(m.Q)
print("pi = ")
print(m.pi)
#
# Generate a couple of realizations
#
print("\nRealization 1 = ")
print(m.realization(10))
print("Realization 2 = ")
print(m.realization(10))
#
# Get the distribution in 10 steps
#
print("\nDistribution of X[9] = ")
print(m.distribution(10))
#
# Get the stationary distribution
#
v=m.stationary_distribution()
print("\nStationary distribution = ")
print(v)
print("Sum of entries in v = ")
print(np.sum(v))
print("\nvQ = ")
print(np.matmul(v,Q))

Q  = 
[[0.00267944 0.15628748 0.35531938 0.35142221 0.13429148]
 [0.21927611 0.22764521 0.25990076 0.24913469 0.04404324]
 [0.22781105 0.26163359 0.14831823 0.17285007 0.18938705]
 [0.19308232 0.12901219 0.20835921 0.32799308 0.1415532 ]
 [0.37314862 0.00466753 0.00423305 0.26285273 0.35509807]]
pi = 
[0.2 0.2 0.2 0.2 0.2]

Realization 1 = 
[0 2 0 4 4 0 3 1 3 1]
Realization 2 = 
[0 3 3 2 4 4 0 2 1 2]

Distribution of X[9] = 
[0.19726916 0.15470546 0.1984887  0.27847816 0.17105852]

Stationary distribution = 
[0.19726916 0.15470547 0.19848871 0.27847816 0.17105851]
Sum of entries in v = 
0.9999999999999998

vQ = 
[0.19726916 0.15470547 0.19848871 0.27847816 0.17105851]


In [22]:
m.Q

array([[0.18156751, 0.4806192 , 0.04165151, 0.21850842, 0.07765336],
       [0.06566476, 0.17723641, 0.06853795, 0.36709564, 0.32146524],
       [0.0957776 , 0.0500638 , 0.22203904, 0.42587178, 0.20624777],
       [0.16663219, 0.3306528 , 0.12239133, 0.23489873, 0.14542495],
       [0.42088728, 0.23331136, 0.13975175, 0.13452431, 0.07152529]])